In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
import pyautogui
import re
from bs4 import BeautifulSoup
import sys
sys.path.append('..')
import constants.constants as const
import constants.file_handler_constants as fh
from constants.restaurant_constants import *

from packages.restaurant.Restaurant import *
from packages.file_handler_package.file_handler import *
from packages.restaurant.wongnai.mulProcess_restaurant_scraping_proxy import *

import os
import glob
import time
import pandas as pd
import numpy as np
from dotenv import load_dotenv, dotenv_values 

from multiprocessing import Pool
from functools import partial

In [ ]:
# *** following this step to use multiprocessing method for restaurant scraping with remote scraping browser
# - select a province by index of 'ALL_REGION_ID_WONGNAI' (manually)
# - find out which is the last page of a province in wongnai.com (manually)

# ** select a province by index of 'ALL_REGION_ID_WONGNAI' (manually)
Idx_of_region = 0
cur_region_data = ALL_REGION_ID_WONGNAI[Idx_of_region]

# ** select first and last page to scrape
first_page = 1
last_page = 5

#
cur_province_en = cur_region_data[0]
cur_province_th = cur_region_data[1]
cur_regionId = cur_region_data[2]

# create list of : [1, 2, 3, ...., last_page]
page_number_list = np.arange(first_page, last_page + 1)
print("check page number list: ")
print(page_number_list)

# get list of dataframe result of all restaurants in a province
# res_allRestaurants_list = []
with Pool(processes=5) as pool: 
    func_param_fix = partial(mulProcess_helper_scrape_restaurants_by_province, province = cur_province_th, wongnai_regionId = cur_regionId)
    res_allRestaurants_list = pool.map(func_param_fix, page_number_list)
    print("end process for province --> %s " % (cur_province_th))

# concat all dataframe into a final dataframe to save in csv.
res_allRestaurants_df = pd.DataFrame()
print("check len of list result dataframe --> ", len(res_allRestaurants_list))
for Idx, cur_df in list(enumerate(res_allRestaurants_list)):
    print("cur_df --> page : ", Idx + first_page)
    print("len cur_df --> ", len(cur_df))
    res_allRestaurants_df = pd.concat([res_allRestaurants_df, cur_df])

# remove duplicate restaurant 
res_allRestaurants_df.drop_duplicates(subset=['name', 'sub_name'], inplace=True)
# set new index
res_allRestaurants_df.set_index(['name', 'sub_name'], inplace=True)


##### save result section

# create directory to store result of scraping restautrant
# for example: 'restaurant_scraping\res_restaurant_scraping\res_restaurant_Mukdahan'
createDirectory(fh.STORE_RESTAURANT_SCRAPING, os.path.join('res_restaurant_scraping', 'res_restaurant_%s' % (cur_province_en)))

# save result dataframe to .csv 
# for example: 'res_restaurant_Mukdahan_page_1_44.csv'
res_file_name = 'res_restaurant_%s_page_%s_%s.csv' % (cur_province_en, first_page, last_page)
res_path = os.path.join(fh.STORE_RESTAURANT_SCRAPING, 'res_restaurant_scraping', 'res_restaurant_%s' % (cur_province_en), res_file_name) 
res_allRestaurants_df.to_csv(res_path, encoding="utf-8")
    

check page number list: 
[1 2 3 4 5]
